In [65]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from typing import TypeAlias
from IPython.display import HTML
%matplotlib inline

Number: TypeAlias = int | float

def number(cin: str) -> Number | None:
    """
    La función number retorna un dato de tipo Number,
    recibe una cadena como parámetro y la función en base a lo recibido,
    retorna un flotante, un entero o en su defecto, retorna None
    """

    result: Number
    if '.' in cin:
        try:
            result = float(cin)
        except ValueError:
            return None

        return result

    try:
        result = int(cin)
    except ValueError:
        return None
    
    return result

def valid(msg: str, value_min: Number | None = None, value_max: Number | None = None, positive: bool = True) -> Number | None:
    """"
    Valida que la entrada de un número esté correcta teniendo en cuenta criterios
    como que tenga que ser un número positivo, el mínimo valor que pueda tomar o
    el máximo valor

    - param msg: str. Es el mensaje que imprime el programa para pedir la entrada
    - param value_min: Number. Es el mínimo valor que puede tomar la entrada (sin incluirlo),
    además, cuando se le es dado el valor None (viene por defecto), no lo toma en cuenta
    - param value_max: Number. Funciona de manera analoga a value_min, con la diferencia
    que se refiere al valor máximo que puede tomar el programa
    - param positive: bool. Quiere decirle al programa que la entrada del número que se desea
    tiene que ser mayor o igual a 0, por defecto está tomado como True

    La función retorna un dato de tipo Number después de haber válidado la entrada
    previamente. En caso de que los parámetros value_min y value_max no estén
    bien planteado (value_min > value_max), la función retorna None
    """

    result: Number | None = None

    if value_min is None and value_max is None:
        while result is None or (result < 0 and positive):
            result = number(input(msg))

            if result is None:
                print('El valor tiene que ser númerico')
                continue

            if result < 0 and positive:
                print('El valor tiene que ser mayor que 0')
        
        return result

    if value_min is None and value_max is not None:
        while result is None or (result < 0 and positive) or result >= value_max:
            result = number(input(msg))

            if result is None:
                print('El valor tiene que ser númerico')
                continue

            if result < 0 and positive:
                print('El valor tiene que ser mayor que 0')
            if result >= value_max:
                print(f'El valor tiene que ser menor que {value_max}')
        
        return result

    if value_max is None and value_min is not None:
        while result is None or (result < 0 and positive) or result <= value_min:
            result = number(input(msg))

            if result is None:
                print('El valor tiene que ser númerico')
                continue

            if result < 0 and positive:
                print('El valor tiene que ser mayor que 0')
            if result <= value_min:
                print(f'El valor tiene que ser mayor que {value_min}')

        return result
    
    if value_min > value_max:
        return result # Retorna None
    
    if value_min == value_max:
        return value_min

    while result is None or (result < 0 and positive) or result <= value_min or result >= value_max:
        result = number(input(msg))

        if result is None:
            print('El valor tiene que ser númerico')
            continue

        if result < 0 and positive:
            print('El valor tiene que ser mayor que 0')
        else:
            if result <= value_min:
                print(f'El valor tiene que ser mayor que {value_min}')
            if result >= value_max:
                print(f'El valor tiene que ser menor que {value_max}')

    return result

In [9]:
force: Number = valid('Rope tension [N]: ', value_min=0)
mass: Number = valid('Rope mass [Kg]: ', value_min=0)
length: Number = valid('Rope length [m]: ', value_min=0)

freq: Number = valid('Frequency [Hz]: ', value_min=0)
amplitude: Number = valid('Amplitude [m]: ', value_min=0)

x0: Number = valid('Initial position [m]: ', positive=False)
v0: Number = valid('Initial velocity [m/s]: ', positive=False)

t: Number = valid('Wave observation time [s]: ') + 1

print('\nUser input')
print(f'Rope tension: {force = } N')
print(f'Rope mass: {mass = } Kg')
print(f'Rope length: {length = } m')
print(f'Frequency: {freq = } Hz')
print(f'Amplitude: {amplitude = } m')
print(f'Initial position: {x0 = } m')
print(f'Initial velocity: {v0 = } m/s')
print(f'Observation time: {t = } s')


User input
Rope tension: force = 0.34 N
Rope mass: mass = 0.001 Kg
Rope length: length = 1.425 m
Frequency: freq = 13.6 Hz
Amplitude: amplitude = 0.5 m
Initial position: x0 = 2 m
Initial velocity: v0 = 0 m/s
Observation time: t = 20 s


In [42]:
linear_density: float = mass/length
speed: float = np.sqrt(force/linear_density)
period: float = 1/freq
angular_freq: float = 2*np.pi*freq
wavelength: float = speed*period
k: float = 2*np.pi / wavelength
p_ave: float = np.sqrt(linear_density*force) * angular_freq**2 * amplitude**2 / 2

print(f'Linear mass density: {linear_density} Kg/m')
print(f'Propagation speed: {speed} m/s')
print(f'Period: {period} s')
print(f'Angular frequency: {angular_freq} rad/s')
print(f'Wavelength: {wavelength} m')
print(f'Average power: {p_ave} J/s')

Linear mass density: 0.0007017543859649122 Kg/m
Propagation speed: 22.01136070305514 m/s
Period: 0.07352941176470588 s
Angular frequency: 85.45132017764237 rad/s
Wavelength: 1.6184824046364072 m
Average power: 14.098716989416259 J/s


In [64]:
fig, ax = plt.subplots()
ax.set_xlim(0, 10)
ax.set_ylim(-amplitude - .1, amplitude + .1)

line1, = ax.plot([], [], lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
ax.grid()

def init():
    line1.set_data([], [])
    time_text.set_text('')
    return line1, time_text

def animate(ti):
    time_text.set_text(f'Time = {ti} s')

    x = np.linspace(0, 20, 1000)
    y = amplitude * np.cos(k*x - angular_freq*ti)
    line1.set_data(x, y)
    return line1,

anim = FuncAnimation(fig, animate, init_func=init,
                     frames=t+1, interval=1./20, blit=True)
# En caso de querer guardarlo, se puede guardar como gif así:
# anim.save('rope_simulation.gif', writer='imagemagick')

plt.close() # Para quitar la gráfica que genera FuncAnimation

HTML(anim.to_jshtml())

In [63]:
fig, ax = plt.subplots()
ax.set_xlim(0, 10)
ax.set_ylim(-1, p_ave*2 + 1)

line1, = ax.plot([], [], lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
ax.grid()

def init():
    line1.set_data([], [])
    time_text.set_text('')
    return line1, time_text

def animate(ti):
    time_text.set_text(f'Time = {ti} s')

    x = np.linspace(0, 20, 1000)
    y = np.sqrt(linear_density*force) * angular_freq**2 * amplitude**2 * np.sin(k*x - angular_freq*ti)**2
    
    line1.set_data(x, y)
    return line1,

anim = FuncAnimation(fig, animate, init_func=init,
                     frames=t+1, interval=1./30, blit=True)

#anim.save('power_rope.gif', writer='imagemagick')

plt.close()
display.HTML(anim.to_jshtml())